In [1]:
import pandas as pd
import pickle
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
Data = pd.read_csv('Dataset/test.csv', header=0)

In [3]:
Data.shape

(1022, 11)

In [4]:
Data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,44,0,0,Yes,Private,Urban,99.34,33.1,never smoked,0
1,Male,52,0,0,Yes,Private,Rural,75.77,30.0,formerly smoked,0
2,Female,0,0,0,No,children,Rural,85.65,17.4,Unknown,0
3,Male,24,0,0,No,Private,Urban,72.29,22.2,Unknown,0
4,Male,48,0,0,Yes,Private,Rural,113.84,21.9,never smoked,0


In [5]:
Data['hypertension'] = Data['hypertension'].astype('object')
Data['heart_disease'] = Data['heart_disease'].astype('object')
Data['age'] = Data['age'].astype('int')
Data['stroke'] = Data['stroke'].astype('object')

In [6]:
Data.dtypes

gender                object
age                    int32
hypertension          object
heart_disease         object
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                object
dtype: object

In [7]:
X_test, y_test = Data.drop(columns=['stroke']), Data['stroke']

In [8]:
# Load the fitted labelencoder from local OS
with open('pickle/OneHotEncoder.pkl', 'rb') as input_file:
    encoder = pickle.load(input_file)
    
# Load the fitted labelencoder from local OS
with open('pickle/MinMaxScaler.pkl', 'rb') as input_file:
    scaler = pickle.load(input_file)
    
# Load the fitted labelencoder from local OS
with open('pickle/PowerTransformer.pkl', 'rb') as input_file:
    transformer = pickle.load(input_file)

In [9]:
# Load the fitted model from local OS
with open('pickle/model.pkl', 'rb') as input_file:
    model = pickle.load(input_file)

In [10]:
# Create our imputer to replace missing values with the mean e.g.
X_test['bmi'].fillna(value=X_test['bmi'].mean(), inplace=True)

In [11]:
#Separate numerical , and categorical variables 

X_test_numeric_data = X_test.select_dtypes(include=[np.number])
X_test_categorical_data = X_test.select_dtypes(exclude=[np.number])

In [12]:
X_test_numeric_data.shape, X_test_categorical_data.shape

((1022, 3), (1022, 7))

In [13]:
X_test_numeric_data = pd.DataFrame(data=transformer.transform(X_test_numeric_data), columns=X_test_numeric_data.columns)
X_test_numeric_data = pd.DataFrame(data=scaler.transform(X_test_numeric_data), columns=X_test_numeric_data.columns)

In [14]:
X_test_categorical_data =pd.DataFrame(data=encoder.transform(X_test_categorical_data), columns=encoder.get_feature_names_out())

In [15]:
pp_X_test = pd.merge(left=X_test_numeric_data, right=X_test_categorical_data, how='inner',
        left_index=True, right_index=True)

In [16]:
y_pred = model.predict(X=pp_X_test)

In [17]:
from sklearn.metrics import classification_report

In [18]:
print(classification_report(y_true=y_test.astype('int').values, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.96      0.95      0.96       974
           1       0.10      0.10      0.10        48

    accuracy                           0.91      1022
   macro avg       0.53      0.53      0.53      1022
weighted avg       0.92      0.91      0.92      1022

